Welcome to this notebook. We'll explore the ATP tennis data records.

The goal is to try to earn money while bidding online. 
If the end model is predicting winning probabilities that are correct, then we can go to a bidding website and compare the «quotes» on the website to the winning probability predicted by the model. And then we bet against any incorrect «quotes». 

Bidders online may underestimate a player's odds, due to a variety of bias in which case we'd bid on him. 

First let's open the CSV, and look a bit a the data

In [2]:
import pandas
import numpy
import matplotlib
import sklearn

In [14]:
tennis_atp_records = pandas.read_csv('ATP.csv')  #, dtype={"draw_size": numpy.int32}, low_memory=False) 
# i had an error due to various datatypes in the  same column. may it's due to the empty values, NaN. low_memory makes it dynamic

/home/francois/.local/share/virtualenvs/CasinoRoyal-anISIRcp/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (3,5,6,7,8,9,12,14,15,16,17,18,19,22,24,25,26,28,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
tennis_atp_records.columns

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'winner_rank', 'winner_rank_points', 'loser_id', 'loser_seed',
       'loser_entry', 'loser_name', 'loser_hand', 'loser_ht', 'loser_ioc',
       'loser_age', 'loser_rank', 'loser_rank_points', 'score', 'best_of',
       'round', 'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon',
       'w_2ndWon', 'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df',
       'l_svpt', 'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved',
       'l_bpFaced'],
      dtype='object')

In [13]:
numpy.mean(tennis_atp_records.draw_size)


TypeError: Could not convert ... to numeric